World map

In [1]:
# 📦 Instalar pacotes
!pip install geopandas matplotlib pyproj

# 📚 Importar pacotes
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from google.colab import files
import zipfile, os

# 📂 Upload do arquivo CSV com os dados
print("📁 Faça upload do arquivo .csv com a coluna 'Country'")
uploaded_csv = files.upload()
csv_path = list(uploaded_csv.keys())[0]
df = pd.read_csv(csv_path)

# 📊 Contagem de frequência por país
country_counts = df['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'Frequency']

# ✅ Calcular porcentagem
total_samples = country_counts['Frequency'].sum()
country_counts['Percentage'] = (country_counts['Frequency'] / total_samples) * 100

# 📂 Upload do shapefile (.zip)
print("📁 Agora faça upload do arquivo .zip com o shapefile do mapa mundial")
uploaded_zip = files.upload()
zip_path = list(uploaded_zip.keys())[0]
extract_dir = "/content/shapefile_data"

# 🗂️ Extrair shapefile
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 🔍 Encontrar o .shp
shp_path = next(os.path.join(extract_dir, f) for f in os.listdir(extract_dir) if f.endswith('.shp'))

# 🌍 Carregar mapa mundial
world = gpd.read_file(shp_path)

# 🔁 Merge com dados de frequência
merge_df = world.merge(country_counts, how='left', left_on='NAME_EN', right_on='Country')
merge_df['Frequency'] = merge_df['Frequency'].fillna(0)
merge_df['Percentage'] = merge_df['Percentage'].fillna(0)

# 🌐 Aplicar projeção de Robinson
merge_df = merge_df.to_crs('+proj=robin')

# 🎨 Criar colormap contínuo pastel: amarelo → verde → azul claro → azul → roxo escuro
pastel_gradient = LinearSegmentedColormap.from_list(
    "YellowToPurplePastel",
    ["#FFFFCC", "#C2E699", "#99D8C9", "#8C96C6", "#6A51A3"],
    N=256
)

# 📊 Separar países com e sem dados
has_data = merge_df[merge_df['Percentage'] > 0]
no_data = merge_df[merge_df['Percentage'] == 0]

# 🗺️ Plotar mapa
fig, ax = plt.subplots(figsize=(16, 9))

# 1. Países sem dados (cinza claro)
no_data.plot(
    ax=ax,
    color="#dadada",
    edgecolor="#f6faff",
    linewidth=0.5
)

# 2. Países com dados (gradiente contínuo)
has_data.plot(
    column='Percentage',
    cmap=pastel_gradient,
    linewidth=0.5,
    edgecolor="#f6faff",
    ax=ax,
    legend=True,
    legend_kwds={
        'label': "Sample Frequency (% of total)",
        'orientation': 'vertical',
        'shrink': 0.6
    }
)

# 🎨 Estética
ax.set_title("Sample Distribution by Country (% of Total)", fontsize=16)
ax.axis('off')

# 💾 Salvar imagem
plt.savefig("world_map_robinson_percentage.png", dpi=600, bbox_inches='tight')
plt.show()
plt.savefig("world_map_transparent_600dpi.png", dpi=600, bbox_inches='tight', transparent=True)
plt.show()
# 📥 Exportar tabela final com frequência e porcentagem
country_counts_sorted = country_counts.sort_values(by='Frequency', ascending=False)
output_path = "/content/sample_frequencies_by_country.csv"
country_counts_sorted.to_csv(output_path, index=False)

# 📎 Oferecer para download
files.download(output_path)

from google.colab import files

# ✅ Baixar as imagens
files.download("world_map_robinson_percentage.png")
files.download("world_map_transparent_600dpi.png")

# ✅ Baixar a planilha
files.download(output_path)


📁 Faça upload do arquivo .csv com a coluna 'Country'


Saving tabela_country_nova.csv to tabela_country_nova.csv


KeyError: 'Country'